In [1]:
import pandas as pd
import numpy as np

import os 
import sys

os.chdir('..') ; sys.path.append(os.getcwd())

In [2]:
Xy = pd.read_parquet('data/feature_matrix_train_v0.parquet')

features = np.loadtxt('data/features.txt', dtype=str)

dtid_ohe = pd.get_dummies(Xy['dtid'])
Xy.drop('dtid', axis=1, inplace=True)
Xy = pd.concat([Xy, dtid_ohe], axis=1)

Xy.head()

,motion,speed,radius,seqnumber,nseq8245,nseq16082,nseq3562,nseq14280,nseq1743,nseq3558,...,5aa2b2e95005741b7d664456,5aafc3cf9e93a13199a1d3e0,5ad494bd9e93a128c8abfca4,5ad49def3c87894334c6d764,5ad7077f9e93a128c85029d7,5af2bdf49e93a128c8cbbc19,5afbe2c59058c264249ac174,5afd4b7c9058c26424f713c5,5b0ef3125005747bb07f99eb,5b115d0b9058c20630b137df
5a497a24cf554f30e846c069,0,1.0,0.0,283,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
5a497a4ce541cd714d45b1c5,0,1.0,0.0,194,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
5a497a4dcf554f30e8472041,0,0.0,0.0,2598,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
5a497aafcf554f30e847cc61,0,0.0,0.0,2721,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
5a497adde541cd714d46accb,0,1.0,0.0,1898,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from utils.evaluation import vincenty_df, vincenty_vec, criterion, plot_error
from geopy.distance import vincenty

In [34]:
X, y = Xy.loc[:, features], Xy.loc[:, ['latitude', 'longitude']]

scaler = StandardScaler() # reuse for test
X_scale = scaler.fit_transform(X)

/home/infres/vcharvet/workspace/pythonProjects/geoloc-challenge/geolocenv/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/infres/vcharvet/workspace/pythonProjects/geoloc-challenge/geolocenv/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [35]:
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_scale)

print(X_pca.shape)

(957567, 289)


In [6]:
%%time
from sklearn.metrics import  make_scorer
from sklearn.model_selection import cross_val_score

custom_loss = make_scorer(criterion, is_df=False)

rf = RandomForestRegressor(n_estimators=500, 
                           max_features='sqrt',
                           max_depth=40,
                           n_jobs=20, verbose=2)

CPU times: user 316 µs, sys: 0 ns, total: 316 µs
Wall time: 338 µs


In [9]:
%%time
cv = cross_val_score(rf, X_scale, y.values, scoring=custom_loss, cv=5, verbose=3, n_jobs=20)

print(cv.mean(), cv.std())

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   2 out of   5 | elapsed: 35.7min remaining: 53.5min


6.9099899943074705 0.8804104690168645
CPU times: user 2.05 s, sys: 11.7 s, total: 13.8 s
Wall time: 43min 14s


[Parallel(n_jobs=20)]: Done   5 out of   5 | elapsed: 43.2min finished


## xgboost

In [18]:
import xgboost 
from sklearn.multioutput import MultiOutputRegressor

In [24]:
%%time
xgb = MultiOutputRegressor(xgboost.XGBRegressor(max_depth=10,n_jobs=24), n_jobs=-1)

cv_xgb = cross_val_score(xgb, X_pca, y.values, scoring=custom_loss, cv=5, verbose=4, n_jobs=24)

print(cv.mean(), cv.std())

[Parallel(n_jobs=24)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 out of   5 | elapsed: 50.5min remaining: 75.7min


6.9099899943074705 0.8804104690168645
CPU times: user 1.28 s, sys: 10.1 s, total: 11.4 s
Wall time: 52min 1s


[Parallel(n_jobs=24)]: Done   5 out of   5 | elapsed: 52.0min finished


## Prediction

In [33]:
pca.components_.shape

(278, 816)

In [36]:
X_test = pd.read_parquet('data/feature_matrix_test_v0.parquet')

dtid_ohe = pd.get_dummies(X_test['dtid'])
X_test.drop('dtid', axis=1, inplace=True)
X_test = pd.concat([X_test, dtid_ohe], axis=1)

X_test_scale = scaler.transform(X_test)
X_test_pca = pca.transform(X_test_scale)

/home/infres/vcharvet/workspace/pythonProjects/geoloc-challenge/geolocenv/lib/python3.6/site-packages/ipykernel_launcher.py:7: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  import sys


In [37]:
%%time
rf = RandomForestRegressor(n_estimators=1000, max_depth=20, max_features='sqrt',
                          n_jobs=20, verbose=3).fit(X_pca, y.values)

# xgb = MultiOutputRegressor(xgboost.XGBRegressor(max_depth=10,n_jobs=24), n_jobs=-1)
# xgb.fit()

[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.


building tree 1 of 1000building tree 2 of 1000
building tree 3 of 1000
building tree 4 of 1000
building tree 5 of 1000
building tree 6 of 1000
building tree 7 of 1000
building tree 8 of 1000
building tree 9 of 1000
building tree 10 of 1000
building tree 11 of 1000
building tree 12 of 1000building tree 13 of 1000

building tree 14 of 1000
building tree 15 of 1000
building tree 16 of 1000building tree 17 of 1000

building tree 18 of 1000
building tree 19 of 1000
building tree 20 of 1000

building tree 21 of 1000
building tree 22 of 1000
building tree 23 of 1000
building tree 24 of 1000
building tree 25 of 1000
building tree 26 of 1000
building tree 27 of 1000
building tree 28 of 1000
building tree 29 of 1000
building tree 30 of 1000
building tree 31 of 1000
building tree 32 of 1000
building tree 33 of 1000
building tree 34 of 1000
building tree 35 of 1000
building tree 36 of 1000
building tree 37 of 1000
building tree 38 of 1000
building tree 39 of 1000
building tree 40 of 1000
building 

[Parallel(n_jobs=20)]: Done  88 tasks      | elapsed:  3.2min


building tree 108 of 1000
building tree 109 of 1000
building tree 110 of 1000
building tree 111 of 1000
building tree 112 of 1000
building tree 113 of 1000
building tree 114 of 1000
building tree 115 of 1000
building tree 116 of 1000
building tree 117 of 1000
building tree 118 of 1000
building tree 119 of 1000
building tree 120 of 1000
building tree 121 of 1000
building tree 122 of 1000
building tree 123 of 1000
building tree 124 of 1000
building tree 125 of 1000
building tree 126 of 1000
building tree 127 of 1000
building tree 128 of 1000
building tree 129 of 1000
building tree 130 of 1000
building tree 131 of 1000
building tree 132 of 1000
building tree 133 of 1000
building tree 134 of 1000
building tree 135 of 1000
building tree 136 of 1000
building tree 137 of 1000
building tree 138 of 1000
building tree 139 of 1000
building tree 140 of 1000
building tree 141 of 1000
building tree 142 of 1000
building tree 143 of 1000
building tree 144 of 1000
building tree 145 of 1000
building tre

[Parallel(n_jobs=20)]: Done 248 tasks      | elapsed:  8.3min


...

[Parallel(n_jobs=20)]: Done 472 tasks      | elapsed: 15.3min


building tree 493 of 1000
building tree 494 of 1000
building tree 495 of 1000
building tree 496 of 1000
building tree 497 of 1000
building tree 498 of 1000
building tree 499 of 1000
building tree 500 of 1000
building tree 501 of 1000
building tree 502 of 1000
building tree 503 of 1000
building tree 504 of 1000
building tree 505 of 1000
building tree 506 of 1000
building tree 507 of 1000
building tree 508 of 1000
building tree 509 of 1000
building tree 510 of 1000
building tree 511 of 1000
building tree 512 of 1000
building tree 513 of 1000
building tree 514 of 1000
building tree 515 of 1000
building tree 516 of 1000
building tree 517 of 1000
building tree 518 of 1000
building tree 519 of 1000
building tree 520 of 1000
building tree 521 of 1000
building tree 522 of 1000
building tree 523 of 1000
building tree 524 of 1000
building tree 525 of 1000
building tree 526 of 1000
building tree 527 of 1000
building tree 528 of 1000
building tree 529 of 1000
building tree 530 of 1000
building tre

[Parallel(n_jobs=20)]: Done 760 tasks      | elapsed: 24.6min


building tree 781 of 1000
building tree 782 of 1000
building tree 783 of 1000
building tree 784 of 1000
building tree 785 of 1000
building tree 786 of 1000
building tree 787 of 1000
building tree 788 of 1000
building tree 789 of 1000
building tree 790 of 1000
building tree 791 of 1000
building tree 792 of 1000
building tree 793 of 1000
building tree 794 of 1000
building tree 795 of 1000
building tree 796 of 1000
building tree 797 of 1000
building tree 798 of 1000
building tree 799 of 1000
building tree 800 of 1000
building tree 801 of 1000
building tree 802 of 1000
building tree 803 of 1000
building tree 804 of 1000
building tree 805 of 1000
building tree 806 of 1000
building tree 807 of 1000
building tree 808 of 1000
building tree 809 of 1000
building tree 810 of 1000
building tree 811 of 1000
building tree 812 of 1000
building tree 813 of 1000
building tree 814 of 1000
building tree 815 of 1000
building tree 816 of 1000
building tree 817 of 1000
building tree 818 of 1000
building tre

[Parallel(n_jobs=20)]: Done 1000 out of 1000 | elapsed: 32.2min finished


In [ ]:
test_pred = pd.DataFrame(rf.predict(X_test_pca), columns=['latitude', 'longitude'])
test_pred['messageid'] = X_test.index

In [42]:
test_pred.to_csv('data/prediction_rf.csv')

In [39]:
print('DONE')

DONE


In [40]:
test_pred.describe()

,latitude,longitude
count,89967.000000,89967.000000
mean,48.876227,2.308530
std,0.044139,0.063993
min,48.526153,1.741678
25%,48.851955,2.281289
50%,48.868848,2.314087
75%,48.890736,2.343436
max,49.547996,2.663995


In [41]:
y.describe()

,latitude,longitude
count,957567.000000,957567.000000
mean,48.878232,2.363743
std,0.123753,0.171150
min,27.029945,-27.022779
25%,48.835434,2.290130
50%,48.876179,2.350454
75%,48.919693,2.429126
max,66.717812,31.491501
